In [55]:
import tweepy
import pandas as pd
import os
import unidecode
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [2]:
sel = pd.read_csv('../Data/SEL/SEL_full.txt', delimiter='\t', encoding= 'ISO-8859-1')

In [3]:
emociones = list(sel['Categoría'].unique())
emociones_dict ={}
for emocion in emociones:
    emociones_dict[emocion] = list(sel[sel['Categoría'] == emocion].sort_values(' PFA',  ascending=False).iloc[0:10]['Palabra'])
    for palabra in emociones_dict[emocion]:
        if not palabra.isascii():
            emociones_dict[emocion].append(unidecode.unidecode(palabra))

In [4]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [69]:
def get_tweets(emotion, words_list, number_tweets, start, end):
    emotion_words = '(' +' OR '.join(words_list) + ')'
    searched_tweets = [tweet for tweet in tweepy.Paginator(client.search_all_tweets, query = emotion_words + ' place_country:CO' ,start_time = start, end_time = end, 
                                max_results=500).flatten(limit=number_tweets)]
    tweets_df = pd.DataFrame(searched_tweets, columns = ['text'])
    tweets_df['emotion']= emotion
    tweets_df['date']= start
    return tweets_df

In [68]:
month = '04'
start_date = datetime.strptime('2021-' + month, '%Y-%m')
end_date = start_date + relativedelta(months=1)
while start_date < end_date:
    print(start_date)
    next_date = start_date + relativedelta(days=1)
    start_date_standard = start_date.isoformat("T") + "Z"
    next_date_standard = next_date.isoformat("T") + "Z"
    for i in emociones_dict.keys():
        print(i)
        df_list.append(get_tweets(i, emociones_dict[i], 100, start_date_standard, next_date_standard ))
        time.sleep(1)
    start_date = next_date
emociones_df = pd.concat(df_list)

Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
